In [1]:
%matplotlib widget

from ogdf_python import *
from itertools import *
from collections import defaultdict
import math
import ipywidgets as widgets

cppinclude("ogdf/basic/simple_graph_alg.h")
cppinclude("ogdf/basic/NodeSet.h")
cppinclude("ogdf/basic/graph_generators/deterministic.h")
cppinclude("./NodePartition.h")

g = cppyy.gbl
vector = g.std.vector
ogdf.setSeed(1)

MatplotlibGraph.curviness = 0.2
MatplotlibGraph.MAX_AUTO_NODE_LABELS = 1001

In [2]:
Color = ogdf.Color
colors = ["#D50000", "#C51162", "#6200EA", "#304FFE", "#00B8D4", "#00BFA5", "#2E7D32", "#AEEA00", "#FFFF00", "#FF6D00", "#DD2C00", "#607D8B",]

BLACK = ogdf.Color()
GRAY = ogdf.Color(128, 128, 128)
RED = ogdf.Color(255, 0, 0)

In [3]:
%%cppdef

using namespace std;
using namespace ogdf;

void bendEdge(GraphAttributes& GA, edge e, double bend) {
	DPoint vec = GA.point(e->target()) - GA.point(e->source());
	DPoint mid = GA.point(e->source()) + (vec / 2);
	DPoint ort = vec.orthogonal();
	ort *= vec.norm() / ort.norm();
	GA.bends(e).pushBack(mid + ort * bend);
}

In [4]:
%%cppdef

using namespace std;
using namespace ogdf;

void randomProperMaximalLevelPlaneGraph(Graph& G, std::vector<std::vector<node>>& emb, int N, int K,
		bool radial) {
	// checks
	OGDF_ASSERT(N > 0);
	OGDF_ASSERT(K > 0);

	// init arrays
	G.clear();
	emb.clear();
	emb.resize(K);

	// create nodes, at least one on each level
	for (int i = 0; i < K; ++i) {
		emb[i].push_back(G.newNode());
	}
	for (int i = K; i < N; ++i) {
		emb[randomNumber(0, K - 1)].push_back(G.newNode());
	}

	// create all edges on each level
	for (int l = 0; l < K - 1; ++l) {
		int lp = 0;
		int up = 0;
		bool wrap_forw = false; // whether there is an edge from emb[l].back() to emb[l+1].front()
		bool wrap_back = false; // whether there is an edge from emb[l].front() to emb[l+1].back()
		while (true) {
			G.newEdge(emb[l][lp], emb[l + 1][up]);

			wrap_forw = wrap_forw || (lp + 1 == emb[l].size() && up == 0);
			wrap_back = wrap_back || (lp == 0 && up + 1 == emb[l + 1].size());

			// select the next node either on the upper or the lower level
			if (lp + 1 < emb[l].size() && up + 1 < emb[l + 1].size()) {
				int r = randomNumber(0, 1);
				lp += r;
				up += 1 - r;
			} else if (lp + 1 < emb[l].size()) {
				lp++;
			} else if (up + 1 < emb[l + 1].size()) {
				up++;
			} else {
				break;
			}
		}

		// add the wrap-around edge
		if (radial) {
			if (wrap_back || (!wrap_forw && randomNumber(0, 1) == 1)) {
				if (!wrap_forw) {
					G.newEdge(emb[l].back(), emb[l + 1].front());
				}
			} else {
				OGDF_ASSERT(!wrap_back);
				G.newEdge(emb[l].front(), emb[l + 1].back());
			}
		}
	}
}

void pruneEdges(Graph& G, int max_edges, int min_deg) {
	vector<edge> edges;
	for (edge e : G.edges) {
		edges.push_back(e);
	}
	mt19937 mt(randomSeed());
	shuffle(edges.begin(), edges.end(), mt);
	for (edge e : edges) {
		if (e->source()->degree() > min_deg && e->target()->degree() > min_deg) {
			G.delEdge(e);
		}
		if (G.numberOfEdges() <= max_edges) {
			break;
		}
	}
}

In [5]:
def drawLevelGraph(GA, emb, scaleX=50, scaleY=50):
    maxlvl = max(map(len, emb))
    for y, lvl in enumerate(emb):
        offs = (maxlvl - len(lvl)) * scaleX / 2
        for x, n in enumerate(lvl):
            GA.x[n] = x * scaleX + offs
            GA.y[n] = y * scaleY

In [6]:
def compute2SATClasses(emb):
    sync = defaultdict(set)
    for i, nodes in enumerate(emb):
        E = [adj.theEdge() for n in nodes for adj in n.adjEntries if adj.isSource()]
        # print(" ".join(map(str,E)))
        for f, s in combinations(E, 2):
            a,b,c,d = f.source(), s.source(), f.target(), s.target()
            if a == b or c == d: continue
            sync[(a,b)].add((c,d))
            sync[(b,a)].add((d,c))
            sync[(c,d)].add((a,b))
            sync[(d,c)].add((b,a))
    #print(sync)
        
    eq = dict()
    cnt = 0
    for a, b in sync:
        if a.index() > b.index():
            a,b = b,a
        if (a,b) in eq: continue
        cnt += 1
        eq[(a,b)] = set()
        eq[(b,a)] = set()
        todo = [(a,b)]
        
        while todo:
            c,d = todo.pop()
            if (a,b) != (c,d):
                if (c,d) in eq: continue
                assert (c,d) not in eq and (d,c) not in eq
            eq[(c,d)] = eq[(a,b)]
            eq[(d,c)] = eq[(b,a)]
            eq[(a,b)].update(sync[(c,d)])
            eq[(b,a)].update(sync[(d,c)])
            for e,f in sync[(c,d)]:
                if (e,f) not in eq:
                    todo.append((e,f))
    #print(eq)

    return sync, eq, cnt

In [7]:
def draw2SATClasses(G, eq , I = None, IA = None):
    I = I or ogdf.GraphCopy(G)
    IA = IA or ogdf.GraphAttributes(I, ogdf.GraphAttributes.all)
    while I.numberOfEdges():
        I.delEdge(I.edges.head())
    for n in G.nodes:
        IA.label[I.copy(n)] = str(n.index())
    for i, nodes in enumerate(emb): # fixme
        for j, n in enumerate(nodes):
            IA.x[I.copy(n)] = math.sin(j / len(nodes) * 2 * math.pi) * 100
            IA.y[I.copy(n)] = math.cos(j / len(nodes) * 2 * math.pi) * -100 + i * 250
    IA.clearAllBends()
    
    added = set()
    edgesets = []
    eqclass = ogdf.EdgeArray[int](I, -1)
    for qq, eqs in eq.items():
        if qq[0].index() > qq[1].index(): continue
        if qq in added: continue
        if (qq[1], qq[0]) in added: continue
        c = colors[len(edgesets) % len(colors)]
        added.update(eqs)
        cur = []
        for a,b in eqs:
            e = I.newEdge(I.copy(a), I.copy(b))
            IA.strokeColor[e] = c
            cur.append(e)
            eqclass[e] = len(edgesets)
        edgesets.append(cur)

    for i, nodes in enumerate(emb):
        for a, b in combinations(nodes, 2):
            e = I.searchEdge(I.copy(a), I.copy(b))
            if not e:
                e = I.newEdge(I.copy(a), I.copy(b))
                c = colors[len(edgesets) % len(colors)]
                IA.strokeColor[e] = c
                eqclass[e] = len(edgesets)
                edgesets.append([e])
                

    return I, IA, edgesets, eqclass

In [8]:
class MatplotlibLevelGraphEditor(MatplotlibGraphEditor):
    def __init__(self, GA, LVL, *args, dist=100, **kwargs):
        self.LVL = LVL
        self.dist = dist
        self.next_line = LVL.size()
        for l, ns in enumerate(LVL.cells()):
            for n in ns:
                GA.y[n] = l*dist
        super().__init__(GA, *args, **kwargs)
        for l in range(LVL.size()):
            self.ax.axhline(l * dist, color="lightgray")

    def update_lvl(self, n):
        l = max(0, round(self.GA.y[n] / self.dist))
        while l >= self.LVL.size():
            self.LVL.newCell()
        while l >= self.next_line:
            self.ax.axhline(self.next_line * self.dist, color="lightgray")
            self.next_line += 1
        self.GA.y[n] = l * self.dist
        if self.LVL.cellOf(n) != l:
            self.LVL.moveToCell(n, l)

    def update_pos(self, n):
        l = self.LVL.cellOf(n)
        pos = len(self.LVL.cell(l)) - 1
        seen = False
        for i, o in enumerate(self.LVL.cell(l)):
            if self.GA.x[o] > self.GA.x[n]:
                pos = i-1 if seen else i
                break
            elif o == n:
                seen = True
        if self.LVL.positionOf(n) != pos:
            self.LVL.moveToPos(n, pos)
                
    def on_node_moved(self, n):
        self.update_lvl(n)
        self.update_pos(n)
        l = self.LVL.cellOf(n)
        for adj in n.adjEntries:
            e = adj.theEdge()
            self.GA.bends[e].clear()
            if self.LVL.cellOf(adj.twinNode()) == l:
                g.bendEdge(self.GA, e, ((((e.index() * 7) % 10) / 50) + 0.2) * ((e.index() % 2) * 2 - 1))

    def add_node(self, n, *args, **kwargs):
        self.on_node_moved(n)
        super().add_node(n, *args, **kwargs)

    def cleared(self):
        super().cleared()
        self.next_line = 1
        self.ax.axhline(0, color="lightgray")


In [9]:
def to_cluster(G, LVL):
    CG = ogdf.ClusterGraph(G)
    for ns in LVL.cells():
        c = CG.newCluster(CG.rootCluster())
        for n in ns:
            CG.reassignNode(n, c)
    return CG

def to_cluster_attrs(GA, LVL):
    CG = to_cluster(GA.constGraph(), LVL)
    CGA = ogdf.ClusterGraphAttributes(CG, GA.attributes())
    ogdf.GraphAttributes.__assign__(CGA, GA)
    return CG, CGA

def from_cluster(CG, LVL):
    assert CG.rootCluster().nodes.empty()
    for i, c in enumerate(CG.rootCluster().children):
        assert c.children.empty()
        if i >= LVL.size():
            LVL.newCell()
        for n in c.nodes:
            LVL.moveToCell(n, i)

In [10]:
G = ogdf.Graph()
GA = ogdf.GraphAttributes(G, ogdf.GraphAttributes.all)
LVL = ogdf.NodePartition(G)
emb = LVL.cells()
lvl = LVL.cellAssignment()
pos = LVL.positionAssignment()

try:
    CG = ogdf.ClusterGraph(G)
    CGA = ogdf.ClusterGraphAttributes(CG, ogdf.ClusterGraphAttributes.all)
    ogdf.GraphIO.read(CGA, CG, G, "counterexample.gml")
    GA.__assign__(CGA)
    from_cluster(CG, LVL)
    #print("Read", CG)
except IOError as e:
    print(e)
    N = [G.newNode() for _ in range(11)]
    E = [G.newEdge(N[u], N[v]) for (u,v) in [
        (0,2),
        (0,7),
        (1,4),
        (1,5),
        (2,8),
        (3,9),
        (7,9),
        (6,8),
        (6,10),
    ]]
    for c, es in enumerate([
        [10, 1, 0],
        [4, 3, 2, 7, 6, 5],
        [9, 8],
    ]):
        if c > 0:
            LVL.newCell()
        for e in es:
            LVL.moveToCell(N[e], c)

# FIXME we assume proper!
drawLevelGraph(GA, emb, 50, 100)

for e in G.edges:
    if lvl[e.source()] > lvl[e.target()]:
        G.reverseEdge(e)

for n in G.nodes:
    GA.label[n] = str(n.index())

widget0 = MatplotlibLevelGraphEditor(GA, LVL)

Read <cppyy.gbl.ogdf.ClusterGraph object at 0x5642c7c4b7f0>


In [11]:
sync, eq, cnt = compute2SATClasses(emb)
I, IA, edgesets, eqclass = draw2SATClasses(G, eq)
widget1 = MatplotlibGraph(IA)

In [12]:
def draw2SATLPClasses(I, IA, G, emb):
    IA2 = ogdf.GraphAttributes(IA)
    drawLevelGraph(IA2, [[I.copy(n) for n in lvl] for lvl in emb], 50, 200)
    dashed_edges = []
    for e in I.edges:
        g.bendEdge(IA2, e, ogdf.randomDouble(0.2, 0.4) * ((e.index() % 2) * 2 - 1))
        IA2.strokeWidth[e] = 2
    for e in G.edges:
        # FIXME keep edges from the beggining to be able to copy GA to IA
        e2 = I.newEdge(e)
        dashed_edges.append(e2)
        IA2.strokeType[e2] = ogdf.StrokeType.Dash
        IA2.strokeColor[e2] = GRAY
        
        IA.strokeColor[e2] = ogdf.Color(255, 255, 255, 0)
        IA.strokeWidth[e2] = 0
        IA.strokeType[e2] = getattr(ogdf.StrokeType, "None")
        
    ILVL = ogdf.NodePartition(I)
    for c, ns in enumerate(LVL.cells()):
        if c > 0:
            ILVL.newCell()
        for n in ns:
            ILVL.moveToCell(I.copy(n), c)

    return IA2, ILVL, dashed_edges

        
IA2, ILVL, dashed_edges = draw2SATLPClasses(I, IA, G, emb)
widget2 = MatplotlibLevelGraphEditor(IA2, ILVL, dist=200)

In [13]:
b_clear = widgets.Button(description="Clear")
b_layout = widgets.Button(description="Layout")
b_reduce = widgets.Button(description="Reduce")

@b_clear.on_click
def clear(b):
    G.clear()
    I.clear()

@b_layout.on_click
def layout(b):
    drawLevelGraph(widget0.GA, widget0.LVL.cells(), 50, widget0.dist)
    widget0.update_all(False)

@b_reduce.on_click
def reduce(b):
    global sync, eq, cnt, I, IA, edgesets, eqclass, IA2, ILVL, dashed_edges
    for e in G.edges:
        if lvl[e.source()] > lvl[e.target()]:
            G.reverseEdge(e)
    I.clear()
    sync, eq, cnt = compute2SATClasses(emb)
    I, IA, edgesets, eqclass = draw2SATClasses(G, eq)
    IA2, ILVL, dashed_edges = draw2SATLPClasses(I, IA, G, emb)
    table.children = make_table()
    widget1.set_graph(IA)
    widget1.update_all(True)
    widget2.LVL = ILVL
    widget2.set_graph(IA2)
    widget2.update_all(True)
    #check(None)

    CG, CGA = to_cluster_attrs(GA, LVL)
    ogdf.GraphIO.write(CGA, "store.gml")

In [14]:
def on_color_change(i, es):
    def fun(c):
        col = ogdf.Color(c['new'])
        colors[i] = col
        for e in es:
            IA.strokeColor[e] = col
            IA2.strokeColor[e] = col
        widget1.update_all(False)
        widget2.update_all(False)
    return fun

def on_revert(i, es, l):
    def fun(b):
        for e in es:
            I.reverseEdge(e)
        l.value = f"{es[0]} +{len(es)-1}" if len(es) > 1 else str(es[0])
        widget1.update_all(False)
        widget2.update_all(False)
    return fun

def on_visibility_change(i, es):
    def fun(c):
        for e in es:
            #IA.strokeType[e] = getattr(ogdf.StrokeType, "None" if c["new"] else "Solid")
            IA.strokeColor[e].alpha(20 if c["new"] else 255)
            IA2.strokeColor[e].alpha(20 if c["new"] else 255)
        widget1.update_all(False)
        widget2.update_all(False)
    return fun

def minmax(seq, key=id):
    it = iter(seq)
    min = max = key(next(it))
    for elem in it:
        elem = key(elem)
        if elem < min:
            min = elem
        if max < elem:
            max = elem
    return min, max

def make_table():
    W = []
    esl = [(i, es, minmax(es, key=lambda e:ILVL.cellOf(e.source()))) for i, es in enumerate(edgesets)]
    esl.sort(key=lambda t: (t[2][0], -t[2][1]))
    for i, es, (min, max) in esl:
        color = IA2.strokeColor[es[0]]
        cp = widgets.ColorPicker(concise=True, disabled=False, value=str(color.toString()))
        cp.observe(on_color_change(i, es), names='value')

        l = widgets.Label((f"{es[0]} +{len(es)-1}" if len(es) > 1 else str(es[0])) + f" [{min}-{max}]")
        
        b1 = widgets.Button(icon="sync")
        b1.on_click(on_revert(i, es, l))
        
        b2 = widgets.ToggleButton(icon="eye-slash")
        b2.observe(on_visibility_change(i, es), names='value')
        b2.value = color.alpha() < 255
        
        W.extend([cp, l, b1, b2])
    return W

table = widgets.GridBox(make_table(), layout=widgets.Layout(grid_template_columns="50px 150px auto auto", width="fit-content"))

In [15]:
label = widgets.Label("???")
label.layout.width = "100px"
b_check = widgets.Button(description="Check")
out = widgets.Output(layout={'border': '1px solid black', 'max_width': '500px', 'max_height': '300px', 'overflow': 'scroll'})

@out.capture()
def check(b):
    hes = ogdf.Graph.HiddenEdgeSet(I)
    for e in dashed_edges:
        hes.hide(e)
    num = ogdf.NodeArray[int](I, -1)
    if ogdf.isAcyclic(I):
        label.value = "Acyclic"
        print("Acyclic")
        ogdf.topologicalNumbering(I, num)
        nodes = sorted(I.nodes, key=num)
        emb2 = [
            [n for n in nodes if lvl[I.original(n)]==l]
            for l in range(len(emb))
        ]
        print(emb2)
        for i, nodes in enumerate(emb2): # fixme
            for j, n in enumerate(nodes):
                IA.x[n] = math.sin(j / len(nodes) * 2 * math.pi) * 100
                IA.y[n] = math.cos(j / len(nodes) * 2 * math.pi) * -100 + i * 250
        drawLevelGraph(IA2, emb2, 50, widget2.dist)
        IA2.clearAllBends()
        ogdf.setSeed(1)
        for e in I.edges:
            IA.strokeWidth[e] = 1
            IA2.strokeWidth[e] = 2
            #if I.isDummy(e.source()) == I.isDummy(e.target()): # others are hidden
            g.bendEdge(IA2, e, ogdf.randomDouble(0.2, 0.4) * ((e.index() % 2) * 2 - 1))
    else:
        label.value = "Cyclic"
        print("Cyclic")
        ogdf.strongComponents(I, num)
        cycles = defaultdict(list)
        for e in I.edges:
            # print(e, num[e.source()], num[e.target()])
            IA.strokeWidth[e] = 1
            IA2.strokeWidth[e] = 2
            if num[e.source()] == num[e.target()]:
                cycles[num[e.source()]].append(e)
        print(cycles)
        cycle = min(cycles.items(), key=lambda i:len(i[1]))[1]
        for e in cycle:
            IA.strokeWidth[e] = 3
            IA2.strokeWidth[e] = 4
    hes.restore()
    widget1.update_all(False)
    widget2.update_all(False)

b_check.on_click(check)
#check(None)

b_shift = widgets.Button(description="Shift Down")

@b_shift.on_click
def do_shift(b):
    cells = list(map(list, LVL.cells()))
    LVL.newCell()
    for i, ns in enumerate(cells):
        for n in ns:
            LVL.moveToCell(n, i + 1)
    drawLevelGraph(GA, LVL.cells(), scaleX=50, scaleY=100)

In [16]:
import os, sh

def write_out(name):
    s = ogdf.GraphIO.SVGSettings()
    s.bezierInterpolation(True)
    s.curviness(0.5)
    os.makedirs("out/tmp", exist_ok=True)

    CG, CGA = to_cluster_attrs(GA, LVL)
    ogdf.GraphIO.write(CGA, "store.gml")
    ogdf.GraphIO.write(CGA, f"out/{name}.gml")
    
    ogdf.GraphIO.drawSVG(GA, "out/tmp/levelplan.svg", s)
    ogdf.GraphIO.drawSVG(IA, "out/tmp/2SATCC.svg", s)
    ogdf.GraphIO.drawSVG(IA2, "out/tmp/2SATCCLP.svg", s)
    
    trans = False
    idx = ogdf.NodeArray[int](G, -1)
    for i, n in enumerate(G.nodes):
        if i != n.index(): trans = True
        idx[n] = i
    
    from contextlib import redirect_stdout
    with open('out/tmp/graph.md', 'w') as f:
        with redirect_stdout(f):
            print("```python")
            print(f"N = [G.newNode() for _ in range({G.numberOfNodes()})]")
            print("E = [G.newEdge(N[u], N[v]) for (u,v) in [")
            for e in G.edges:
                print(f"\t({idx[e.source().index()]}, {idx[e.target().index()]}),")
            print("]]")            
            print("for c, ns in enumerate([")
            for L in emb:
                print(f"\t{[idx[n.index()] for n in sorted(L, key=lambda n: IA2.x[I.copy(n)])]},")
            print("]):")
            print("\twhile c >= LVL.size(): LVL.newCell()")
            print("\tfor n in ns: LVL.moveToCell(N[n], c)")
            if trans:
                print(f"for i, l in {[(i, n.index()) for i, n in enumerate(G.nodes) if i != n.index()]}:")
                print("\tGA.label[N[i]] = str(l)")
            print("```")
    
    _sh = sh.bake(_cwd="out/tmp", _long_prefix="-", _env={"PATH": os.environ["PATH"]})
    _sh.pandoc("graph.md", o="graph.pdf")
    SVGS = ["levelplan", "2SATCC", "2SATCCLP"]
    for s in SVGS:
        _sh.rsvg_convert(s + ".svg", f="pdf", o=s + ".pdf")
    _sh.pdfunite("graph.pdf", *[s+".pdf" for s in SVGS], f"../{name}.pdf")

i_fname = widgets.Text(value='reduction', placeholder='Filename')
b_write = widgets.Button(description="Save", icon="save", layout=widgets.Layout(width = "100px"))
b_read = widgets.Button(description="Load", icon="folder-open", layout=b_write.layout)

@b_write.on_click
def do_write(b):
    write_out(i_fname.value)

@b_read.on_click
def do_read(b):
    CG = ogdf.ClusterGraph(G)
    CGA = ogdf.ClusterGraphAttributes(CG, ogdf.ClusterGraphAttributes.all)
    ogdf.GraphIO.read(CGA, CG, G, f"out/{i_fname.value}.gml")
    GA.__assign__(CGA)
    from_cluster(CG, LVL)
    print("Read", CG)

In [17]:
widget0.ax.margins(0.05, 0.2)
widget2.ax.margins(0.05, 0.2)
box = widgets.HBox([
    widget0.ax.figure.canvas,
    #widget1.ax.figure.canvas,
    widget2.ax.figure.canvas,
    widgets.VBox([
        widgets.HBox([b_clear, b_layout, b_reduce]),
        widgets.HBox([i_fname, b_write, b_read]),
        table,
        widgets.HBox([label, b_check, b_shift]),
        out
    ])
])
box